In [82]:
import csv
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale,robust_scale,minmax_scale,maxabs_scale
import datetime
from dateutil.parser import parse
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
from scipy import stats


In [83]:
data='./training.csv'
train=pd.read_csv(data,header=0)
train=pd.DataFrame(data=train)


In [84]:
train=train.drop(["VehYear"],axis=1)
train=train.drop(["RefId"],axis=1)
train=train.drop(["Model"],axis=1)
train=train.drop(["Trim"],axis=1)
train=train.drop(["SubModel"],axis=1)
train=train.drop(["WheelTypeID"],axis=1)
train=train.drop(["PRIMEUNIT"],axis=1)
train=train.drop(["AUCGUART"],axis=1)
train=train.drop(["VNZIP1"],axis=1)
train=train.drop(["PurchDate"],axis=1)
train=train.drop(["BYRNO"],axis=1)


In [85]:
train.loc[[15906, 24567, 24578, 70432, 70434, 70437, 70445, 70446, 70450],["Transmission"]]="AUTO"

train.dropna(subset=["Color"], inplace=True)
train.dropna(subset=["MMRAcquisitionAuctionAveragePrice"], inplace=True)
train.dropna(subset=["MMRCurrentAuctionAveragePrice"], inplace=True)
train.dropna(subset=["Size"], inplace=True)
train.loc[train["WheelType"].isnull(),"WheelType"]="Alloy"
train.loc[train['Transmission']=="Manual","Transmission"]="MANUAL"


In [86]:
train_x=train.ix[:,1:-1]
train_y=train.ix[:,0]


In [87]:
train_x1=train_x.copy()
train_y1=train_y.copy()

In [88]:
num_col=['MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
                        'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
                        'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
                        'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice']


In [89]:
train_x1["AuctionAve"]=sum(train_x1[a] for a in num_col)/len(num_col)
train_x1=train_x1.drop(num_col,axis=1)

    

In [90]:
train_x1=pd.concat([train_y1,train_x1],axis=1)

In [91]:
from sklearn.cross_validation import train_test_split
def split_train_test(df):
    temp = df.drop("IsBadBuy", axis=1)
    train_X, test_X, train_Y, test_Y = train_test_split(temp, df.IsBadBuy,
                                                        test_size=0.3, random_state=4531)
    train_X = pd.DataFrame(train_X, columns=df.columns[1:])
    test_X = pd.DataFrame(test_X, columns=df.columns[1:])
    train_Y = pd.DataFrame(train_Y, columns=['IsBadBuy'])
    test_Y = pd.DataFrame(test_Y, columns=['IsBadBuy'])


    return train_X, test_X, train_Y, test_Y

In [92]:
train_x1, test_x1, train_y1, test_y1=split_train_test(train_x1)


In [93]:
train_x1["VehicleAge"]=(train_x1["VehicleAge"]-(train_x1["VehicleAge"].mean()))/float(train_x1["VehicleAge"].std())
train_x1["VehOdo"]=(train_x1["VehOdo"]-(train_x1["VehOdo"].mean()))/float(train_x1["VehOdo"].std())
train_x1["VehBCost"]=(train_x1["VehBCost"]-(train_x1["VehBCost"].mean()))/float(train_x1["VehBCost"].std())
train_x1["AuctionAve"]=(train_x1["AuctionAve"]-(train_x1["AuctionAve"].mean()))/float(train_x1["AuctionAve"].std())
    



In [94]:
test_x1["VehicleAge"]=(test_x1["VehicleAge"]-(test_x1["VehicleAge"].mean()))/float(test_x1["VehicleAge"].std())
test_x1["VehOdo"]=(test_x1["VehOdo"]-(test_x1["VehOdo"].mean()))/float(test_x1["VehOdo"].std())
test_x1["VehBCost"]=(test_x1["VehBCost"]-(test_x1["VehBCost"].mean()))/float(test_x1["VehBCost"].std())
test_x1["AuctionAve"]=(test_x1["AuctionAve"]-(test_x1["AuctionAve"].mean()))/float(test_x1["AuctionAve"].std())

In [95]:
train_x1=pd.get_dummies(train_x1)


In [97]:
test_x1=pd.get_dummies(test_x1)

In [98]:
from sklearn import metrics, cross_validation
from sklearn.linear_model import LogisticRegression



logreg=LogisticRegression()

predicted = cross_validation.cross_val_predict(logreg, train_x1, train_y1, cv=10)
scores = cross_validation.cross_val_score(logreg, train_x1, train_y1, cv=10)
print metrics.classification_report(train_y1, predicted) 
print(metrics.accuracy_score(train_y1, predicted))


print('average score: {}'.format(scores.mean()))

IndexError: too many indices for array